## 0. Inicjalizacja

#### 0.1 Import

In [ ]:
import pandas as pd
import os
import sys
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from scipy.stats import uniform
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    HalvingGridSearchCV,
    HalvingRandomSearchCV,
    train_test_split,
)
from time import time
from scipy.stats import randint
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json

sys.path.append(os.path.abspath("../../"))
from src.utils import load_train_test_data, get_params, SEPHORA_DATASET
from src.pipeline import build_pipeline

#### 0.2 Parametry oraz Dataset

In [ ]:
dataset_name = "sephora"

In [ ]:
common_params, custom_params = get_params(dataset_name)

In [ ]:
training_data, _ = load_train_test_data("data/train_test_data", dataset_name)
training_data = training_data[custom_params["pipeline"]["selected"]]
training_data = training_data[0:1_000]
X_train, X_val, y_train, y_val = train_test_split(
    training_data.drop(columns=[custom_params["features"]["label"]], axis=1),
    training_data[custom_params["features"]["label"]],
    test_size=0.2,
    random_state=1
)

#### 0.3 Pipeline

In [ ]:
rf_pipeline = build_pipeline(
    custom_params, dataset_name, "all", RandomForestClassifier(n_jobs=-1, random_state=1)
)
svm_pipeline = build_pipeline(custom_params, dataset_name, "all", SVC(random_state=1))

#### 0.4 Funkcja do eksperymentów

In [ ]:
def run_search(name, searcher, X_train, X_val, y_train, y_val):
    print("Fit...")
    start = time()
    searcher.fit(X_train, y_train)
    end = time()

    elapsed = end - start
    best_model = searcher.best_estimator_
    n_models = len(searcher.cv_results_["mean_test_score"])
    best_params = searcher.best_params_
    best_score = searcher.best_score_

    print("Predict...")
    y_pred = best_model.predict(X_val)

    results = {
        "Method": name,
        "Best f1_weighted (CV)": best_score,
        "Accuracy": accuracy_score(y_val, y_pred),
        "Precision": precision_score(
            y_val, y_pred, average="weighted", zero_division=0
        ),
        "Recall": recall_score(y_val, y_pred, average="weighted", zero_division=0),
        "F1 (weighted)": f1_score(y_val, y_pred, average="weighted", zero_division=0),
        "Time (s)": elapsed,
        "Models Tried": n_models,
        "Best Params": best_params,
        "Best Estimator": best_model,
    }

    return searcher, results

#### 0.5 Parametry do eksperymentów

In [ ]:
rf_distributions = {
    "classifier__n_estimators": randint(30, 60),
    "classifier__max_depth": randint(10, 50),
    "classifier__criterion": ["gini", "entropy"],
    "classifier__min_samples_leaf": randint(1, 7),
    "classifier__max_features": ["sqrt", None],
}

svm_distributions = {
    "classifier__C": uniform(0.5, 1.5),
    "classifier__kernel": ["linear", "rbf", "poly"],
    "classifier__degree": [2, 3],
    "classifier__gamma": ["scale", "auto"],
}

In [ ]:
rf_param_grid = {
    "classifier__n_estimators": [30, 60],
    "classifier__max_depth": [10, 30, 50],
    "classifier__criterion": ["gini", "entropy"],
    "classifier__min_samples_leaf": [1, 5],
    "classifier__max_features": ["sqrt", None],
}

svm_param_grid = {
    "classifier__C": [0.8, 1, 1.5],
    "classifier__kernel": ["linear", "rbf", "poly"],
    "classifier__degree": [2, 3],
    "classifier__gamma": ["scale", "auto"],
}

## 1. RandomSearch

#### 1.1 Opis działania, możliwości, ograniczenia

**[RandomizedSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)** losowo wybiera kombinacje hiperparametrów z podanego rozkładu lub listy wartości. Można ustawić liczbę iteracji, co pozwala kontrolować czas trwania wyszukiwania.

- Dzięki losowości jest szybszy od GridSearch, lecz przeszukuje mniejszą przestrzeń hiperparametrów.
Może wciąż znaleźć bardzo dobre parametry przy mniejszej złożoności obliczeniowej

- Wynik zależy od losowości, ponieważ może pominąć najlepsze hiperparametry.
- Wyniki mogą się różnić przy wielu próbach uruchomienia przez losowość

#### 1.2 Jakość najlepszego modelu oraz czas działania

In [ ]:
clf = RandomizedSearchCV(
    rf_pipeline,
    rf_distributions,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
    n_iter=40,
)
_, rf_randomized_results = run_search(
    "RandomizedSearchCV", clf, X_train, X_val, y_train, y_val
)

In [ ]:
clf = RandomizedSearchCV(
    svm_pipeline,
    svm_distributions,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
    n_iter=40,
)
_, svm_randomized_results = run_search(
    "RandomizedSearchCV", clf, X_train, X_val, y_train, y_val
)

#### 1.3 Liczba iteracji metody

Liczba iteracji jest równa parametrowi n_iter.

In [ ]:
print(f"RandomizedSearchCV wykonał {rf_randomized_results['Models Tried']} pomiarów")

## 2. GridSearch

#### 2.1 Opis działania, możliwości, ograniczenia

[GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) przeszukuje wybrane przestrzenie hiperparametrów tworząc iloczyn kartezjański na podanych wartościach hiperparametrów. 

- Dzięki temu bardzo precyzyjnie możemy oszacować, które wartości nas potencjalnie najbardziej interesują oraz możemy bardzo dokładnie przeszukać przestrzeń hiperaparametrów.
- Deterministyczny oraz bardzo dokładny

- Bardzo wymagający obliczeniowo, dokonanie tylu eksperymentów ile podano wartości oraz list wartości.
- Bardzo wymgający czasowo.

#### 2.2 Jakość najlepszego modelu oraz czas działania

In [ ]:
clf = GridSearchCV(
    rf_pipeline,
    rf_param_grid,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
)
_, rf_grid_results = run_search("GridSearchCV", clf, X_train, X_val, y_train, y_val)

In [ ]:
clf = GridSearchCV(
    svm_pipeline,
    svm_param_grid,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
)
_, svm_grid_results = run_search("GridSearchCV", clf, X_train, X_val, y_train, y_val)

#### 2.3 Liczba iteracji metody

Liczba pomiarów jest równa iloczynowi liczebności każdej z list hiperparametrów

In [ ]:
print(f"GridSearchCV wykonał {rf_grid_results['Models Tried']} pomiarów")

## 3. HalvingGridSearch

#### 3.1 Opis działania, możliwości, ograniczenia

[HalvingGridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) działa analogicznie do GridSearch'a, lecz optymalizuje swoje działanie poprzez wykonywanie eksperymentów na części danych. Na początku testuje wiele kombinacji na małych próbkach danych, a w kolejnych iteracjach zwiększa ilość danych tylko dla najlepszych kandydatów.

- Posida zalety grid searcha dokładnego przeszukania przestrzeni hiperparametrów przy mniejszym nakładzie mocy obliczeniowej oraz krótszym czasie wykonywania.
- Utrzymuje dokładność GridSearch przy mniejszych kosztach.

- Wadą jest fakt, iż jest to eksperymentalna funkcja sklearn, czyli nie jest ustabilizowana/może się zmieniać.
- Wciąż przeszukujemy ogromną liczbę zbiorów hiperparametrów

#### 3.2 Jakość najlepszego modelu oraz czas działania

In [ ]:
clf = HalvingGridSearchCV(
    rf_pipeline,
    rf_param_grid,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
)
_, rf_halv_grid_results = run_search(
    "HalvingGridSearchCV", clf, X_train, X_val, y_train, y_val
)

In [ ]:
clf = HalvingGridSearchCV(
    svm_pipeline,
    svm_param_grid,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
)
_, svm_halv_grid_results = run_search(
    "HalvingGridSearchCV", clf, X_train, X_val, y_train, y_val
)

#### 3.3 Liczba iteracji metody

In [ ]:
print(f"GridSearchCV wykonał {rf_halv_grid_results['Models Tried']} pomiarów")

## 4. HalvingRandomSearch

#### 4.1 Opis działania, możliwości, ograniczenia

[HalvingRandomSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) jest połączeniem HalvingGridSearch oraz RandomSeach. Działa z mechanizmem losowego wyboru hiperparametrów z aspektem odsiewania słabszych i trenowania pozostałych na większej liczbie danych.

Najbardziej efektywny czasowo i najmniej wymagający obliczeniowo

Ponownie jest eksperymentalną funkcja sklearn oraz tak jak RandomSearch może pominąć najlepsze wartości hiperparametrów

#### 4.2 Jakość najlepszego modelu oraz czas działania

In [ ]:
clf = HalvingRandomSearchCV(
    rf_pipeline,
    rf_param_grid,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
    n_candidates=40,
)
_, rf_halv_rand_grid_results = run_search(
    "HalvingRandomSearchCV", clf, X_train, X_val, y_train, y_val
)

In [ ]:
clf = HalvingRandomSearchCV(
    svm_pipeline,
    svm_param_grid,
    return_train_score=True,
    scoring="f1_weighted",
    n_jobs=-1,
    n_candidates=40,
)
_, svm_halv_rand_grid_results = run_search(
    "HalvingRandomSearchCV", clf, X_train, X_val, y_train, y_val
)

#### 4.3 Liczba iteracji metody

In [ ]:
print(f"GridSearchCV wykonał {rf_halv_rand_grid_results['Models Tried']} pomiarów")

## 5. Podsumowanie

#### 5.1 Porównanie modeli

In [ ]:
rf_results = [
    rf_randomized_results,
    rf_grid_results,
    rf_halv_grid_results,
    rf_halv_rand_grid_results,
]
df_rf_results = pd.DataFrame(rf_results)

In [ ]:
svm_results = [
    svm_randomized_results,
    svm_grid_results,
    svm_halv_grid_results,
    svm_halv_rand_grid_results,
]
df_svm_results = pd.DataFrame(svm_results)

In [ ]:
plt.figure(figsize=(15, 8))
metrics_to_plot = [
    "Best f1_weighted (CV)",
    "Accuracy",
    "Precision",
    "Recall",
    "F1 (weighted)",
]
df_melted = df_rf_results.melt(
    id_vars="Method", value_vars=metrics_to_plot, var_name="Metric", value_name="Score"
)

sns.barplot(data=df_melted, x="Metric", y="Score", hue="Method")
plt.title("Porównanie metryk jakości modeli")
plt.xticks(rotation=45)
plt.legend(title="Metoda")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_rf_results, x="Method", y="Time (s)", palette="muted", hue="Method")
plt.title("Czas działania dla każdej metody")
plt.ylabel("Czas (s)")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

In [ ]:
df_rf_results["Time per Model"] = (
    df_rf_results["Time (s)"] / df_rf_results["Models Tried"]
)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    data=df_rf_results,
    x="Method",
    y="Time per Model",
    hue="Method",
    palette="viridis",
    legend=False,
)
plt.title("Średni czas trenowania pojedynczego modelu")
plt.ylabel("Czas (s) na model")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

In [ ]:
rf_max_index = df_rf_results["Best f1_weighted (CV)"].idxmax()
svm_max_index = df_svm_results["Best f1_weighted (CV)"].idxmax()
rf_best_params = df_rf_results["Best Params"][rf_max_index]
svm_best_params = df_svm_results["Best Params"][svm_max_index]
best_params = {
    "RandomForest": rf_best_params,
    "SVM": svm_best_params,
}

with open(f"data/models/{dataset_name}/best_params.json", "w") as f:
    json.dump(best_params, f)

#### 5.2 Porównanie wyników

Najlepsze wyniki osiągnął model Randomized Search

Model trenował się najwięcej czasu oraz posiada najgorszą metrykę względem czasu/model

Pozostałe metody osiągnęły bardzo podobne wyniki, lecz różniły je czasy wykonywania.

Zakładająć, iż wyniki bybły nieznacznie podobne kolejny model z najlepszymi wynikami posiada HalvingRandomSearch ze względu na definitywnie najkrótszy czas działania.

#### 5.3 Wybrów i uzasadnienie

Nie ma jednego dobrego wyjścia, są one bardzo zależne od naszych potrzeb i kompetencji.

Najlepszą opcją, gdy potrzebujemy szybko znaleźć "dobry" model jest zdecydowanie HalvingRandomSearch ze względu na akceptowalne wyniki w zaskakująco krótkim czasie. (Dobre wyniki, szybkie działanie).

Najlepszym wyborem, gdy znamy dokładnie model, dane oraz posiadamy dużą wiedzę dziedzinową i intuicję jest GridSearch ze względu na możliwość dokładnego zbadania wielu hiperparametrów dla naszej hipotezy. (Najlepszy wynik, najdłużej się wykonuje).

Najlepszym wyborem, gdy nie do końca znamy potencjalne hiperparametry oraz nie zależy nam na czasie jest RandomizedSearch dzięki któremu w sposób losowy możemy osiąganąć dobre wyniki dla modelu oraz samemu zdecydować ilu iteracji potrzebujemy.

Dla mnie, jako studenta bez silnej intuicji odnośnie nieznanych klasyfikatorów i dla nieznanyc zbiorów danych najlepszą opcją byłby HalvingRandomSearch, ponieważ umożliwiłby mi w szybki sposób zrozumieć dla jakich mniej więcej parametrów model działałby dobrze.